**Week4 part 2 Limitation(s) of sklearn’s non-negative matrix factorization library**

In [9]:
import pandas as pd
import numpy as np
from scipy.sparse import coo_matrix
from sklearn.decomposition import NMF

In [24]:
import warnings
warnings.filterwarnings('ignore')

csv files have been downloaded from coursera week 3 assignments and uploaded to my github repo so we can read_csv from http link

In [10]:
users = pd.read_csv('https://raw.githubusercontent.com/gt2onew/dtsa5510/main/week4/users.csv')
movies = pd.read_csv('https://raw.githubusercontent.com/gt2onew/dtsa5510/main/week4/movies.csv')
train = pd.read_csv('https://raw.githubusercontent.com/gt2onew/dtsa5510/main/week4/train.csv')
test = pd.read_csv('https://raw.githubusercontent.com/gt2onew/dtsa5510/main/week4/test.csv')

In [11]:
users.head(3)

,uID,gender,age,accupation,zip
0,1,F,1,10,48067
1,2,M,56,16,70072
2,3,M,25,15,55117


In [12]:
movies.head(3)

,mID,title,year,Doc,Com,Hor,Adv,Wes,Dra,Ani,...,Chi,Cri,Thr,Sci,Mys,Rom,Fil,Fan,Act,Mus
0,1,Toy Story,1995,0,1,0,0,0,0,1,...,1,0,0,0,0,0,0,0,0,0
1,2,Jumanji,1995,0,0,0,1,0,0,0,...,1,0,0,0,0,0,0,1,0,0
2,3,Grumpier Old Men,1995,0,1,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0


In [13]:
train.head(3)

,uID,mID,rating
0,744,1210,5
1,3040,1584,4
2,1451,1293,5


In [14]:
test.head(3)

,uID,mID,rating
0,2233,440,4
1,4274,587,5
2,2498,454,3


1. Load the movie ratings data (as in the HW3-recommender-system) and use matrix factorization technique(s) and predict the missing ratings from the test data. Measure the RMSE.

In [15]:
allusers = list(users['uID'])
allmovies = list(movies['mID'])
mid2idx = dict(zip(movies.mID,list(range(len(movies)))))
uid2idx = dict(zip(users.uID,list(range(len(users)))))
ind_movie = [mid2idx[x] for x in train.mID] 
ind_user = [uid2idx[x] for x in train.uID]
rating_train = list(train.rating)
Mr = np.array(coo_matrix((rating_train, (ind_user, ind_movie)), shape=(len(allusers), len(allmovies))).toarray())


In [16]:
Mr

array([[5, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [3, 0, 0, ..., 0, 0, 0]])

In [17]:
Mr.shape

(6040, 3883)

In [19]:
type(Mr)

numpy.ndarray

In [72]:
# Computing the sparsity
len(Mr.nonzero()[0]) / float(Mr.shape[0] * Mr.shape[1])

0.029852745794625237

In [73]:
model = NMF(n_components=20)     
W = model.fit_transform(Mr)
H = model.components_

In [74]:
W.shape

(6040, 20)

In [75]:
H.shape

(20, 3883)

In [76]:
ind_movie_test = [mid2idx[x] for x in test.mID] 
ind_user_test = [uid2idx[x] for x in test.uID]
rating_test = list(test.rating)
Mr_test = np.array(coo_matrix((rating_test, (ind_user_test, ind_movie_test)), shape=(len(allusers), len(allmovies))).toarray())

In [77]:
Mr_test

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [78]:
Mr_test.shape

(6040, 3883)

In [79]:
len(Mr_test.nonzero()[0]) / float(Mr_test.shape[0] * Mr_test.shape[1])

0.012794052185362243

In [80]:
Mr_pred = H.T.dot(W.T).T

In [81]:
Mr_pred

array([[1.85616566e+00, 5.51756056e-01, 9.48810332e-03, ...,
        7.29404396e-03, 3.55254959e-03, 6.97319363e-02],
       [8.13584471e-01, 4.11828523e-01, 7.23517988e-02, ...,
        1.99408913e-02, 0.00000000e+00, 5.21014089e-02],
       [9.04407588e-01, 1.34726861e-01, 1.14007191e-02, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       ...,
       [6.79105963e-01, 1.94617574e-02, 4.15181147e-03, ...,
        1.37846721e-03, 1.78014529e-03, 7.76960417e-03],
       [1.42875757e+00, 2.84369309e-01, 1.21148619e-01, ...,
        3.89885024e-02, 0.00000000e+00, 0.00000000e+00],
       [1.48810941e+00, 9.14776807e-02, 3.39576452e-03, ...,
        9.16802288e-02, 1.07363041e-01, 4.68595647e-01]])

In [82]:
from sklearn.metrics import mean_squared_error


In [83]:
rmse = np.sqrt(mean_squared_error(Mr_pred[Mr_test.nonzero()].flatten(), Mr_test[Mr_test.nonzero()].flatten()))

In [84]:
rmse

2.858502602473767

2. Discuss the results and why they did not work well compared to simple baseline or similarity-based methods we’ve done in Module 3. Can you suggest a way(s) to fix it?



*   The result rmse 2.86 did not look good, simple baseline is around 1.26 rmse per week3 assignment
*   NMF didn't take true movie and user features into account, but purely based on user-movie rating matrix
*   Also NMF default l2 loss does not work well for sparse matrix which has lots of 0
*   Tuning n_components may be helpful to improve the performance through gridsearchcv; And using Kl loss is good for data that has lots of 0



